# Import necessary packages

In [79]:
import pandas as pd
import numpy as np
import nltk
import future
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords

In [80]:
#TQDM is a progress bar library with good support for nested loops and Jupyter/IPython notebooks.
from tqdm import tqdm

In [81]:
# The Natural Language Toolkit, or more commonly NLTK, is a suite of libraries and programs for symbolic and 
# statistical natural language processing for English written in the Python programming language.
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from bs4 import BeautifulSoup
import re

In [82]:
from keras.utils import to_categorical
import random
# from tensorflow import set_random_seed
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential

In [83]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/wangxiaolin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/wangxiaolin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wangxiaolin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Import Data

The dataset is merged using the methods below:
 - Follow these steps or run the shell script here: Preprocessing Script
 - Move the tar file to the directory where you want this data to be stored.
 - Open a terminal window and cd to the directory that you put aclImdb_v1.tar.gz in.gunzip -c aclImdb_v1.tar.gz | tar xopf -
 - cd aclImdb && mkdir movie_data 
 - for split in train test; do for sentiment in pos neg; do for file in $split/$sentiment/*; do cat $file >> movie_data/full_${split}.txt; echo >> movie_data/full_${split}.txt; done; done; done;

In [105]:
# Read data
train_pos = pd.read_csv('/Users/wangxiaolin/Documents/HKUST Machine Learning/project/dataset/train_pos.csv')
train_neg = pd.read_csv('/Users/wangxiaolin/Documents/HKUST Machine Learning/project/dataset/train_neg.csv')
test_pos = pd.read_csv('/Users/wangxiaolin/Documents/HKUST Machine Learning/project/dataset/test_pos.csv')
test_neg = pd.read_csv('/Users/wangxiaolin/Documents/HKUST Machine Learning/project/dataset/test_neg.csv')

In [106]:
# Add one column
train_pos["sentiment"] = "1"
train_neg["sentiment"] = "0"
test_pos["sentiment"] = "1"
test_neg["sentiment"] = "0"

In [107]:
# combine the train positive and negative  
train = [train_pos, train_neg]
train = pd.concat(train, ignore_index = True)

# combine the test positive and negative  
test = [test_pos, test_neg]
test = pd.concat(test, ignore_index = True)

In [108]:
train.head()

,Unnamed: 0,Review,sentiment
0,0,Bromwell High is a cartoon comedy. It ran at t...,1
1,1,Homelessness (or Houselessness as George Carli...,1
2,2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,3,This is easily the most underrated film inn th...,1
4,4,This is not the typical Mel Brooks film. It wa...,1


# Data Pre-processing

In order to do Natural Language Processing tasks, we need to understand and be able to use regular expressions. The data needs to be cleaned before they are vecterized. Therefore, 
 - the html content should be removed, 
 - the non-alphabetic characters should be removed, 
 - the sentences should be tokenized 
 - and the word should be lemmatized to its lemma.

In [109]:
def clean_data(data):
    reviews = []

    for sent in tqdm(data):
        
        #remove html content
        review_text = BeautifulSoup(sent).get_text()
        
        #remove non-alphabetic characters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
        #tokenize the sentences
        words = word_tokenize(review_text.lower())
    
        #lemmatize each word to its lemma
        lemma_words = [lemmatizer.lemmatize(i) for i in words]
    
        reviews.append(lemma_words)

    return(reviews)

In [110]:
#generate the cleaned data by calling the function above
train["Review"] = clean_data(train["Review"])
test["Review"] = clean_data(test["Review"])
print(len(train["Review"]))
print(len(test["Review"]))

100%|██████████| 25000/25000 [01:36<00:00, 260.41it/s]


25000
25000


# Vectorization

The data has to be converted into numeric representation so that the algorithm could understand them. This conversion is so called "vectorization" or "one hot encoding".

Generally, we could created a large matrix with only one column for every unique word in the dataset (here, the reviews). Then the words in the review are transformed into one row containing 0s and 1s. Here, 1 stands for the fact that the word appears in this piece of review. We would expect each row of the matrix would contain mostly zeros.

In [111]:
train_review = train.Review

In [112]:
test_review = test.Review

In [113]:
for i in range(len(test_review)):
    test_review[i]=' '.join(test_review[i])
for i in range(len(train_review)):
    train_review[i]=' '.join(train_review[i])

/Users/wangxiaolin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/wangxiaolin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [114]:
train_sentiment = train.sentiment
test_sentiment = test.sentiment

In [120]:
# TFIDF Vecterization
tfidf = TfidfVectorizer(min_df=5) #minimum document frequency of 5
train_tfidf = tfidf.fit_transform(train_review)

# Fit the model - logistic regression

In [121]:
# We seperate the training set and the validation set
X_train, X_val, y_train, y_val = train_test_split(
    train_tfidf, train.sentiment, train_size = 0.75
)

In [122]:
# We select the best tuning parameter fist using the validation set for logistic regression model
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.80096
Accuracy for C=0.05: 0.8368
Accuracy for C=0.25: 0.87552
Accuracy for C=0.5: 0.88768
Accuracy for C=1: 0.89232


In [125]:
# According to the results above, we choose C = 1
lr = LogisticRegression(C=1)
lr.fit(train_tfidf, train_sentiment)
lr_predictions = lr.predict(tfidf.transform(test_review))
accuracy_score(predictions, test_sentiment)
print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(predictions, test_sentiment)))
metrics.confusion_matrix(predictions, test_sentiment)

Accuracy for C=1: 0.882


array([[11041,  1491],
       [ 1459, 11009]])

In [65]:
# We print the performance report for this model
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [126]:
lr_report=classification_report(test.sentiment,lr_predictions,target_names=['Positive','Negative'])
print(lr_report)

              precision    recall  f1-score   support

    Positive       0.88      0.88      0.88     12500
    Negative       0.88      0.88      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



Interpretation of the results:
- we obtained an accuracy score of 0.882
- Here, A precision of 0.88 suggests that among all reviews predicted to be positive, 88% of them are truly positive. 
- A recall of 0.88 indicates that among all truly positive claims, 88% of them are retrieved by the model.

# Fit the model - Decision tree

In [68]:
from sklearn.tree import DecisionTreeClassifier

In [127]:
dt=DecisionTreeClassifier()
dt.fit(train_tfidf, train_sentiment)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [128]:
print ("Final Accuracy: %s" 
       % accuracy_score(test.sentiment, dt.predict(tfidf.transform(test_review))))

Final Accuracy: 0.70552


In [129]:
# We print the performance report for this model
dt_predict = dt.predict(tfidf.transform(test_review))

In [130]:
dt_report=classification_report(test.sentiment,dt_predict,target_names=['Positive','Negative'])
print(dt_report)

              precision    recall  f1-score   support

    Positive       0.71      0.71      0.71     12500
    Negative       0.71      0.71      0.71     12500

    accuracy                           0.71     25000
   macro avg       0.71      0.71      0.71     25000
weighted avg       0.71      0.71      0.71     25000



Interpretation of the results:
- we obtained an accuracy score of 0.70552
- Here, A precision of 0.71 suggests that among all reviews predicted to be positive, 71% of them are truly positive.
- A recall of 0.71 indicates that among all truly positive claims, 71% of them are retrieved by the model.

# Fit the model - Random Forest

In [74]:
from sklearn.ensemble import RandomForestClassifier

In [131]:
rf=RandomForestClassifier()
rf.fit(train_tfidf, train_sentiment)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [132]:
print ("Final Accuracy: %s" 
       % accuracy_score(test.sentiment, rf.predict(tfidf.transform(test_review))))

Final Accuracy: 0.8382


In [133]:
# We print the performance report for this model
rf_predict = rf.predict(tfidf.transform(test_review))
rf_report=classification_report(test.sentiment,rf_predict,target_names=['Positive','Negative'])
print(rf_report)

              precision    recall  f1-score   support

    Positive       0.83      0.85      0.84     12500
    Negative       0.85      0.82      0.84     12500

    accuracy                           0.84     25000
   macro avg       0.84      0.84      0.84     25000
weighted avg       0.84      0.84      0.84     25000



Interpretation of the results:
- we obtained an accuracy score of 0.8382
- Here, A precision of 0.83 suggests that among all reviews predicted to be positive, 83% of them are truly positive.
- A recall of 0.85 indicates that among all truly positive claims, 85% of them are retrieved by the model.

# Conclusion

The accuracy scores for logistic regression model, decision tree model and random forest model are 0.882, 0.70552, and 0.8382 respectively. Therefore, the logistic regression model performs the best on the dataset according to the accuracy score. Same conclusion could be reached by checking the F1-score.